In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

# !pip install --upgrade --force-reinstall --no-deps kaggle
# import os
# os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Data/Kaggle API Token/"

# root_path = "/content/drive/MyDrive/Data/Plant Pathology"
# os.chdir(root_path)

# !kaggle competitions download -c plant-pathology-2021-fgvc8
# !unzip plant-pathology-2021-fgvc8.zip  && rm plant-pathology-2021-fgvc8.zip


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import os
img_size = (224,224)
img_height =224
batch_size = 128
root_path = "/content/drive/MyDrive/Data/Plant Pathology"

In [3]:
df = pd.read_csv(os.path.join(root_path,'train.csv'))
df

,image,labels
0,800113bb65efe69e.jpg,healthy
1,8002cb321f8bfcdf.jpg,scab frog_eye_leaf_spot complex
2,80070f7fb5e2ccaa.jpg,scab
3,80077517781fb94f.jpg,scab
4,800cbf0ff87721f8.jpg,complex
...,...,...
18627,fffb900a92289a33.jpg,healthy
18628,fffc488fa4c0e80c.jpg,scab
18629,fffc94e092a59086.jpg,cider_apple_rust
18630,fffe105cf6808292.jpg,scab frog_eye_leaf_spot


In [5]:
df['labels'].value_counts()  

scab                               4826
healthy                            4624
frog_eye_leaf_spot                 3181
cider_apple_rust                   1860
complex                            1602
powdery_mildew                     1184
scab frog_eye_leaf_spot             686
scab frog_eye_leaf_spot complex     200
frog_eye_leaf_spot complex          165
rust frog_eye_leaf_spot             120
rust complex                         97
powdery_mildew complex               87
Name: labels, dtype: int64

In [4]:
filepath_prefix = os.path.join(root_path,"train_images/")
df['absolute_path'] = filepath_prefix + df.image
df.head()

,image,labels,absolute_path
0,800113bb65efe69e.jpg,healthy,/content/drive/MyDrive/Data/Plant Pathology/tr...
1,8002cb321f8bfcdf.jpg,scab frog_eye_leaf_spot complex,/content/drive/MyDrive/Data/Plant Pathology/tr...
2,80070f7fb5e2ccaa.jpg,scab,/content/drive/MyDrive/Data/Plant Pathology/tr...
3,80077517781fb94f.jpg,scab,/content/drive/MyDrive/Data/Plant Pathology/tr...
4,800cbf0ff87721f8.jpg,complex,/content/drive/MyDrive/Data/Plant Pathology/tr...


In [ ]:
# from sklearn.model_selection import train_test_split
# X_train, X_val = train_test_split(df,test_size=0.25,stratify=df['labels'],
#                                   random_state=2,shuffle=True)

In [5]:
from keras.preprocessing.image import ImageDataGenerator
rescale_value = 1/255.

data_gen = ImageDataGenerator(rescale=rescale_value, validation_split=0.2)

In [6]:
train_generator = data_gen.flow_from_dataframe(
                  df,
                  directory = None,
                  x_col = "absolute_path",
                  y_col = "labels",
                  target_size= img_size,
                  batch_size=batch_size,
                  class_mode='categorical',
                  shuffle=True,
                  subset='training',
                  validate_filenames=False)

Found 14906 non-validated image filenames belonging to 12 classes.


In [7]:
validation_generator = data_gen.flow_from_dataframe(
                  df,
                  directory = None,
                  x_col = "absolute_path",
                  y_col = "labels",
                  target_size=img_size,
                  batch_size=batch_size,
                  class_mode='categorical',
                  shuffle=False,
                  subset='validation',
                  validate_filenames=False)

Found 3726 non-validated image filenames belonging to 12 classes.


In [8]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Conv2D, MaxPool2D, Flatten
import pickle
from tensorflow import keras

from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(
    patience = 15,
    min_delta = 0.001,
    restore_best_weights = True
)

In [9]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13850681522441215886
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14674281152
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18008790860005666342
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


#Model-1

In [10]:
import tensorflow_hub as hub
import tensorflow as tf

feature_extractor = "https://tfhub.dev/google/imagenet/inception_resnet_v2/classification/4"
feature_extractor_layer = hub.KerasLayer(feature_extractor,input_shape=(img_height,img_height,3))
feature_extractor_layer.tranable = False

model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(12,activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')
history = model.fit(train_generator,batch_size=batch_size,epochs=2,validation_data=validation_generator,callbacks=[early_stopping])

Epoch 1/2
 95/117 [=======================>......] - ETA: 25:13 - loss: 2.6847 - accuracy: 0.3204

In [ ]:
if os.path.exists(os.path.join(root_path,"model-01")):
  model = keras.models.load_model(os.path.join(root_path,"model-01"))
if not os.path.exists(os.path.join(root_path,"model-01")):
  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')
  history = model.fit(train_generator,batch_size=batch_size,epochs=1,validation_data=validation_generator,callbacks=[early_stopping])
  model.save(os.path.join(root_path,"model-01"))
  with open(os.path.join(root_path,"model-01-history"),'wb') as file_pi:
    pickle.dump(history.history,file_pi)

  8/117 [=>............................] - ETA: 3:46:21 - loss: 4.8112 - accuracy: 0.1479

In [ ]:
_, eval_acc = model.evaluate(train_generator, verbose=0)
print(f"In train evaluation: {eval_acc}\n")
_, eval_acc = model.evaluate(validation_generator, verbose=0)
print(f"Best accuracy in evaluation: {eval_acc}\n")

In [ ]:
history = pickle.load(open(os.path.join(root_path,"model-01-history"), "rb"))
import matplotlib.pyplot as plt

plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
l=os.listdir(os.path.join(root_path,"test_images"))
sub = pd.DataFrame({'image':l})
sub

,image
0,85f8cb619c66b863.jpg
1,ad8770db05586b59.jpg
2,c7b03e718489f3ca.jpg


#Model -02